In [2]:
import sys
sys.path.append(sys.path[0].replace('/notebooks',''))

In [3]:
from dataset import DatasetGeneral
import torch
from torch import nn
from torchvision.models import mobilenet_v2
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, InterpolationMode, Resize, CenterCrop, Normalize, PILToTensor, ConvertImageDtype, ToTensor

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import cv2
from PIL import Image

from tqdm import tqdm

import os

In [162]:
model = mobilenet_v2(weights='IMAGENET1K_V1')
model = torch.nn.Sequential(*(list(model.children())[:-1]))

In [7]:
model = model.train()

In [8]:
from utils import get_images_labels
from sklearn.model_selection import train_test_split

In [9]:
df = get_images_labels("/home/lucas/Documentos/SamplesImages/",format_img='jpeg')

Images and labels loaded


In [11]:
map_labels = {j:i for i,j in enumerate(df['images_labels'].unique())}
df['images_labels'] = df['images_labels'].apply(lambda x: map_labels[x])

In [8]:
train, aux = train_test_split(df,test_size=0.3,random_state=69,stratify=df['images_labels'])
val, test = train_test_split(aux,test_size=0.5,random_state=69,stratify=aux['images_labels'])

In [9]:
workers = 0
batch_size = 2

transform = Compose([
        Resize(256),
        CenterCrop(224),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    ])

print("------- Starting dataloaders -------")

train_dataset   = DatasetGeneral(train['images_paths'].tolist(), train['images_labels'].tolist(), transform_data=transform,pos_neg_examples=4)
val_dataset     = DatasetGeneral(val['images_paths'].tolist(), val['images_labels'].tolist(), transform_data=transform,pos_neg_examples=4)
test_dataset    = DatasetGeneral(test['images_paths'].tolist(), test['images_labels'].tolist(), transform_data=transform,pos_neg_examples=4)

train_dataloader = DataLoader(train_dataset, num_workers=workers, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, num_workers=workers, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, num_workers=workers, batch_size=batch_size)

------- Starting dataloaders -------


In [10]:
triplet_loss = torch.nn.TripletMarginLoss(margin=1.0, p=2)

In [24]:
model = model.train()
for anchor, positive, negative in tqdm(train_dataloader,total=train_dataloader.__len__()):
    anchor_pred = model(anchor)
    positive_pred = [model(x) for x in positive]
    negative_pred = [model(x) for x in negative]
    
    loss = triplet_loss(anchor_pred,positive_pred,negative_pred)
    loss.backward()
    break

  0%|                                                                                         | 0/50400 [00:00<?, ?it/s]/home/lumalfa/Face-Similarity/dataset.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(anchor), torch.stack(positive), torch.stack(negative)
  0%|                                                                                         | 0/50400 [00:11<?, ?it/s]


TypeError: triplet_margin_loss(): argument 'positive' (position 2) must be Tensor, not list

# Testando o triplet loss

In [94]:
transform = Compose([
        Resize(256),
        CenterCrop(224),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    ])


In [213]:
imgs = [transform(Image.fromarray(cv2.cvtColor(cv2.imread(x),cv2.COLOR_BGR2RGB))).unsqueeze(0) for x in list(df['images_paths'])]

In [214]:
imgs = torch.concat(imgs)

In [215]:
imgs.shape

torch.Size([9, 3, 224, 224])

In [220]:
output = model(imgs)

In [221]:
output.shape

torch.Size([9, 1280, 7, 7])

In [218]:
output = 

In [185]:
output.shape

torch.Size([9, 1280, 7, 7])

In [1]:
anchor = output[:3]
positive = torch.stack([output[:3],output[:3],output[:3]])
negative = torch.stack([output[2:5],output[2:5],output[2:5]])

anchor = torch.nn.functional.adaptive_avg_pool2d(anchor, 1).reshape(-1,anchor.shape[1])
positive = torch.nn.functional.adaptive_avg_pool2d(positive, 1).reshape(positive.shape[0],positive.shape[1],-1)
negative = torch.nn.functional.adaptive_avg_pool2d(negative, 1).reshape(negative.shape[0],negative.shape[1],-1)

NameError: name 'output' is not defined

In [ ]:
anchor

In [263]:
torch.clamp(torch.nn.functional.pairwise_distance(anchor[0],positive[0]) - torch.nn.functional.pairwise_distance(anchor[0],negative[0]) +1,0)

tensor([0.0000, 1.9020, 1.4389], grad_fn=<ClampBackward1>)

In [113]:
torch.stack([anchor[0]]*positive[0].shape[0]).shape

torch.Size([3, 1280, 7, 7])

In [118]:
positive[0:1,...].shape

torch.Size([1, 3, 1280, 7, 7])

In [19]:
positive.unsqueeze(0).shape

torch.Size([1, 2, 4, 3, 224, 224])

In [9]:
anchor = torch.rand(100,1,1028)
positive = torch.rand(1028)
negative = torch.rand(1028)

In [12]:
a = torch.concat([positive,negative])

In [17]:
torch.sqrt(torch.sum(torch.square(a)))

tensor(25.8865)

In [33]:
anchor_pred.unsqueeze(0).shape

torch.Size([1, 2, 1280, 7, 7])

In [29]:
triplet_loss(anchor_pred.unsqueeze(0),torch.stack(positive_pred),torch.stack(negative_pred))

RuntimeError: The size of tensor a (2) must match the size of tensor b (4) at non-singleton dimension 1